# Use CSP to solve Sudoku problem

**FOLLOW THE INSTRUCTION! IF I CANNOT READ IT, I CANNOT GRADE IT!**

If you want to modify the function (name, parameters, etc.), put comments there and explain your logic!


## Create puzzle


In [2]:
from random import sample
def create_game(base=3, side = 9):
  # pattern for a baseline valid solution
  def pattern(r,c):
    return (base*(r%base)+r//base+c)%side

  # randomize rows, columns and numbers (of valid base pattern)
  def shuffle(s):
    return sample(s,len(s))

  def remove_cells(board, ratio = 0.75):
    squares = side*side
    empties = int(squares * ratio)
    for p in sample(range(squares),empties):
        board[p//side][p%side] = 0

    numSize = len(str(side))
    return board

  rBase = range(base)
  rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ]
  cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
  nums  = shuffle(range(1,base*base+1))

  # produce board using randomized baseline pattern
  board = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
  return remove_cells(board)

board = create_game()

def print_board(board):
  for row in board:
    print(" ".join(map(str, row)))

print_board(board)

6 1 0 0 0 0 0 4 0
0 4 0 0 0 0 0 0 0
0 0 2 0 0 7 0 0 5
0 0 0 9 0 3 0 0 7
0 2 0 6 0 0 0 0 0
0 0 9 7 2 0 0 0 0
0 0 0 0 0 0 0 7 1
8 0 0 0 6 0 0 0 0
2 0 0 0 0 0 0 0 3


## Sudoku CSP Definition

**Variables:** Open Cells C(x, y) for x, y in range(1, 9).  
**Domains:** Possible Values D(x) for x in range(1, 9).  
**Constraints:**
*   Each *row* must have exactly 1 of each value.
*   Each *column* must have exactly 1 of each value.
*   Each *3x3* square must have exactly 1 of each value.
*   Every cell must be filled with a non-zero number.

## Backtracking

Solve with Backtracking algorithm

If you do not like the following function design, please feel free to modify it. Add comments to explain the logic.

In [1]:
def getUnsiganedVariable( board ):
  """
  Gets the first unassigned variable on the board

  :param board: Game board to get variable from
  :return: Coordinates of unassigned variable (tuple) or failure (-1)
  """
  # Initialize return variable with failure state
  var = -1

  # Search rows...
  for x in range( 9 ):
    # Search columns...
    for y in range( 9 ):
      # If we find an empty spot,
      if board[ x ][ y ] == 0:
        return (x, y) # Return the empty spot

  # Otherwise, return failure
  return var

In [3]:
def isValidAssignment( board, cur_pos, digit ):
  """
  Checks the board to see if an assignment is valid

  :param board: Game board to check
  :param cur_pos: Coordinates of position to check
  :param digit: Digit to check
  :return: Valid (bool)
  """
  # Initialize return variable
  valid = False

  # Shortcut coordinates
  x = cur_pos[ 0 ]
  y = cur_pos[ 1 ]

  # Currently valid digits for this location
  valid_digits = [ 1, 2, 3, 4, 5, 6, 7, 8, 9 ]

  # Check row and column if for validity
  for i in range( 9 ):
    if board[ i ][ y ] in valid_digits:
      valid_digits.remove( board[ i ][ y ] )
    if board[ x ][ i ] in valid_digits:
      valid_digits.remove( board[ x ][ i ] )

  # Check local 3x3 square for validity
  for i in range(3):
        for j in range(3):
            if(board[i + x - x%3][j + y - y%3] in valid_digits):
                valid_digits.remove( board[i + x - x%3][j + y - y%3] )

  # If digit remains in valid_digits, the move is valid
  if digit in valid_digits:
    valid = True

  return valid

In [4]:
import time

def backtrackingSolver(board):
  """
  Recursive backtracking sudoku solver

  :param board: Game board to check
  :return: Solution (board) or Failure (bool)
  """
  # Get next open board slot
  pos = getUnsiganedVariable( board )

  # If no more open slots, board is solved
  if pos == -1:
    return board

  # Check each digit available
  for i in range(1, 10):
    # Check if digit is valid placement
    if isValidAssignment( board, pos, i ):
      # If valid, assign to board
      board[pos[0]][pos[1]] = i

      # If next slot fills successfully, pass back up
      if backtrackingSolver( board ) != False:
        return board

      # Otherwise reset the slot and try next digit
      board[pos[0]][pos[1]] = 0

  # If slot can't be filled, return failure
  return False

In [7]:
board = create_game()

startTime = time.time()
board = backtrackingSolver(board)
print( "Completed in: " + str( round( time.time()-startTime, 3 ) ) + " seconds." )

print_board(board)

Completed in: 31.456 seconds.
3 5 6 7 4 8 1 9 2
8 4 7 2 1 9 5 3 6
1 9 2 6 5 3 4 8 7
2 1 9 4 3 6 8 7 5
6 3 4 5 8 7 9 2 1
7 8 5 1 9 2 3 6 4
5 2 3 8 6 4 7 1 9
9 7 1 3 2 5 6 4 8
4 6 8 9 7 1 2 5 3


## Forward Checking + Minimum Remaining Values

Please feel free to modify the function design. Add comments to explain the logic.

In [8]:
def createDomains(board):
  """
  Creates valid domains for each cell of a game board.

  :param board: The game board to create domains for
  :return: 2D array of domains (list)
  """
  # Initialize the domains as a 2D array
  domains = {}

  # Iterate over the rows
  for x in range( 9 ):
    # Iterate over the cols
    for y in range( 9 ):
      # Create a full domain for the current cell
      localDomain = [ 1, 2, 3, 4, 5, 6, 7, 8, 9 ]

      # Remove the current location from domains
      if board[x][y] in localDomain:
        localDomain.remove( board[x][y] )

      # Remove any entries in current row/col from domain
      for i in range( 9 ):
        if board[x][i] in localDomain:
          localDomain.remove( board[x][i] )
        if board[i][y] in localDomain:
          localDomain.remove( board[i][y] )

      # Calculate the entries in the square at current position
        for i in range(3):
          for j in range(3):
              if(board[i + x - x%3][j + y - y%3] in localDomain):
                  localDomain.remove( board[i + x - x%3][j + y - y%3] )

      # Assign finished local domain to final domain list
      domains[ ( x, y ) ] = localDomain

  return( domains )

In [9]:
def isValidAssignment(cur_pos, digit, domains):
  """
  Checks an assignment to see if it is valid.

  :param cur_pos: Tuple of current coordinates.
  :param digit:   Digit (int) to check placement of.
  :param domains: List of domains for board.
  :return: Assignment (bool) indicating validity
  """
  # Initialize return variable
  assignment = False

  # If the digit is within the domain of current position
  if( digit in domains[ cur_pos ] ):
    # It is a valid digit
    assignment = True

  # Returns the boolean
  return assignment

In [10]:
def getUnsignedVariable( domains ):
  """
  Gets an unassigned variable with the least remaining values.

  :param domains: The domains for the current game board.
  :return: Dictionary of best value coords and domain.
  """
  # Initialize domain to be returned - biggest possible domain
  val = [ 1,2,3,4,5,6,7,8,9 ]
  # Initialize coordinates. -1 indicates failure
  best = -1

  # Iterates over the coordinates in domains
  for cord in domains:
    # If the cord is empty and the domains is smaller than current
    if( ( len( domains[ cord ] ) < len( val ) ) and
        ( board[ cord[ 0 ] ][ cord[ 1 ] ] == 0 ) ):
      # Update new smallest domain
      val = domains[ cord ]
      # Update new best cord
      best = cord

  # Return best coord and domain
  return { best: val }

In [11]:
def forwardChecking( domain ):
    """
    Forward checking game board solver.

    :param board: Current state of the board
    :param domain: Domains for each position on the board
    :return: Solved (bool)
    """
    # If the board contains no empty positions
    if all( all( cell != 0 for cell in row ) for row in board ):
        # All variables are assigned, solution found
        return True

    # Get info for position with LRV
    currentVal = getUnsignedVariable( domain )
    currentPos = list( currentVal.keys() )[ 0 ]
    currentDom = list( currentVal.values() )[ 0 ]

    # If the current position has valid moves
    if currentDom:
        # Iterate over the possible moves
        for value in currentDom:
            # Get updated info (in case domain changed)
            currentDom = list( getUnsignedVariable( domain ).values() )[ 0 ]

            # Check for a valid assignment from domain
            if isValidAssignment( currentPos, value, domain ):
                # Assign value and update domain
                board[ currentPos[ 0 ] ][ currentPos[ 1 ] ] = value
                newDomain = createDomains( board )

                # Forward check the next position
                if forwardChecking( newDomain ):
                    return True

                # If the current assignment didn't lead to a solution, backtrack
                board[ currentPos[ 0 ] ][ currentPos[ 1 ] ] = 0

    # No valid assignment found, return False
    return False

In [12]:
import time

def FC_solver(board):
  """
  Solves a game board using forward checking

  :param board: The game board to create domains for
  :return: Solved board (or unsolved if solution not possible)
  """
  # Start timer and create domains
  startTime = time.time()
  domains = createDomains(board)

  # val holds possible values of the cell
  assignment = {(i, j): val for (i, j), val in domains.items()}

  if forwardChecking(assignment):
    print( "Completed in: " + str( round( time.time()-startTime, 3 ) ) + " seconds." )
    return board
  return board


In [30]:
board = create_game()
FC_solver(board)

Completed in: 1.846 seconds.


[[4, 9, 6, 8, 2, 7, 3, 1, 5],
 [5, 7, 2, 1, 3, 9, 6, 8, 4],
 [1, 3, 8, 4, 6, 5, 9, 7, 2],
 [9, 1, 7, 5, 4, 2, 8, 3, 6],
 [2, 4, 3, 9, 8, 6, 1, 5, 7],
 [6, 8, 5, 7, 1, 3, 4, 2, 9],
 [3, 5, 1, 2, 9, 4, 7, 6, 8],
 [8, 2, 4, 6, 7, 1, 5, 9, 3],
 [7, 6, 9, 3, 5, 8, 2, 4, 1]]